<h1 style="text-align: center; color:rgb(254, 255, 255);">Insider Trades Analysis</h1>

### Automatic data collection with github actions

Scrape SEC form 4 insider purchases over $500k

In [3]:
import pandas as pd
def display_insider_trades():
    # Read the insider buys CSV file
    df = pd.read_csv('Scraping/insider_buys.csv')
    
    # Display the dataframe
    display(df)
display_insider_trades()

,Filing Date,Ticker,Company Name,Transaction Price,Price Bought,Value
0,2025-02-07 21:14:34,PBF,Pbf Energy Inc.,28.70,27.820000,4305060
1,2025-02-07 20:45:00,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770
2,2025-02-07 20:30:26,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770
3,2025-02-07 20:30:05,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770
4,2025-02-07 19:54:28,AKRO,"Akero Therapeutics, Inc.",48.00,54.209999,9600000
5,2025-02-07 19:35:56,KEY,Keycorp /New/,17.85,17.840000,892500
6,2025-02-07 17:01:57,FLWS,1 800 Flowers Com Inc,8.04,8.060000,2933822
7,2025-02-07 16:21:19,TECX,"Tectonic Therapeutic, Inc.",50.68,36.470001,39999977
8,2025-02-07 16:18:52,EL,Estee Lauder Companies Inc,65.80,65.040001,5724373
9,2025-02-07 16:04:52,MRK,"Merck & Co., Inc.",88.50,87.305000,1327500


Calculate the change in price of the stock in 24h, 1 week and 1 month after the filing date.

In [16]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os

def calculate_price_changes():
    # Read the insider buys CSV file
    df = pd.read_csv('Scraping/insider_buys.csv')
    
    # Convert Filing Date to datetime
    df['Filing Date'] = pd.to_datetime(df['Filing Date'])
    
    # Initialize new columns
    df['24h_Change_%'] = None
    df['1w_Change_%'] = None
    df['1m_Change_%'] = None
    
    current_time = datetime.now()
    
    # Suppress yfinance warnings
    import logging
    logging.getLogger('yfinance').setLevel(logging.CRITICAL)
    
    for idx, row in df.iterrows():
        try:
            ticker = row['Ticker']
            filing_date = row['Filing Date']
            base_price = row['Price Bought']
            
            if pd.isna(base_price):
                continue
                
            stock = yf.Ticker(ticker)
            
            # Calculate target dates keeping exact hour and minute
            h24_target = filing_date + timedelta(days=1)
            w1_target = filing_date + timedelta(days=7)
            m1_target = filing_date + timedelta(days=30)
            
            # Check if filing was after market hours (after 16:00)
            is_after_hours = filing_date.hour >= 16
            
            # Only calculate if the time has passed
            if h24_target < current_time:
                h24_data = stock.history(
                    start=(h24_target - timedelta(days=1)).strftime('%Y-%m-%d'),
                    end=(h24_target + timedelta(days=1)).strftime('%Y-%m-%d'),
                    interval='1m'
                )
                if not h24_data.empty:
                    h24_data.index = h24_data.index.tz_localize(None)
                    
                    # If filing was after hours, check if we have data for the next trading day
                    if is_after_hours:
                        next_day_data = h24_data[h24_data.index.date > filing_date.date()]
                        if next_day_data.empty:
                            continue  # Skip if no next day data available
                    
                    closest_time = h24_data.index[h24_data.index.get_indexer([h24_target], method='nearest')[0]]
                    target_price = h24_data.loc[closest_time]['Close']
                    change = round(((target_price - base_price) / base_price) * 100, 2)
                    df.at[idx, '24h_Change_%'] = f"{'+' if change >= 0 else ''}{change}%"
            
            if w1_target < current_time:
                w1_data = stock.history(
                    start=(w1_target - timedelta(days=1)).strftime('%Y-%m-%d'),
                    end=(w1_target + timedelta(days=1)).strftime('%Y-%m-%d'),
                    interval='1m'
                )
                if not w1_data.empty:
                    w1_data.index = w1_data.index.tz_localize(None)
                    closest_time = w1_data.index[w1_data.index.get_indexer([w1_target], method='nearest')[0]]
                    target_price = w1_data.loc[closest_time]['Close']
                    change = round(((target_price - base_price) / base_price) * 100, 2)
                    df.at[idx, '1w_Change_%'] = f"{'+' if change >= 0 else ''}{change}%"
            
            if m1_target < current_time:
                m1_data = stock.history(
                    start=(m1_target - timedelta(days=1)).strftime('%Y-%m-%d'),
                    end=(m1_target + timedelta(days=1)).strftime('%Y-%m-%d'),
                    interval='1m'
                )
                if not m1_data.empty:
                    m1_data.index = m1_data.index.tz_localize(None)
                    closest_time = m1_data.index[m1_data.index.get_indexer([m1_target], method='nearest')[0]]
                    target_price = m1_data.loc[closest_time]['Close']
                    change = round(((target_price - base_price) / base_price) * 100, 2)
                    df.at[idx, '1m_Change_%'] = f"{'+' if change >= 0 else ''}{change}%"
                    
        except Exception:
            continue
    
    # Handle existing analysis file
    analysis_file = 'insider_buys_analysis.csv'
    if os.path.exists(analysis_file):
        existing_df = pd.read_csv(analysis_file)
        existing_df['Filing Date'] = pd.to_datetime(existing_df['Filing Date'])
        
        # Combine existing and new data
        combined_df = pd.concat([existing_df, df])
        
        # Remove duplicates based on Filing Date, Ticker, and Transaction Price
        combined_df = combined_df.drop_duplicates(
            subset=['Filing Date', 'Ticker', 'Transaction Price'], 
            keep='last'
        )
        
        # Sort by Filing Date (most recent first)
        combined_df = combined_df.sort_values('Filing Date', ascending=False)
        
        # Save combined data
        combined_df.to_csv(analysis_file, index=False)
    else:
        # If no existing file, save new data
        df.to_csv(analysis_file, index=False)
    
    return df

# Execute the analysis
analysis_df = calculate_price_changes()
pd.set_option('display.max_rows', None)
display(analysis_df)

,Filing Date,Ticker,Company Name,Transaction Price,Price Bought,Value,24h_Change_%,1w_Change_%,1m_Change_%
0,2025-02-07 21:14:34,PBF,Pbf Energy Inc.,28.70,27.820000,4305060,None,None,None
1,2025-02-07 20:45:00,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770,None,None,None
2,2025-02-07 20:30:26,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770,None,None,None
3,2025-02-07 20:30:05,TKO,"Tko Group Holdings, Inc.",163.27,166.410004,54181770,None,None,None
4,2025-02-07 19:54:28,AKRO,"Akero Therapeutics, Inc.",48.00,54.209999,9600000,None,None,None
5,2025-02-07 19:35:56,KEY,Keycorp /New/,17.85,17.840000,892500,None,None,None
6,2025-02-07 17:01:57,FLWS,1 800 Flowers Com Inc,8.04,8.060000,2933822,None,None,None
7,2025-02-07 16:21:19,TECX,"Tectonic Therapeutic, Inc.",50.68,36.470001,39999977,None,None,None
8,2025-02-07 16:18:52,EL,Estee Lauder Companies Inc,65.80,65.040001,5724373,None,None,None
9,2025-02-07 16:04:52,MRK,"Merck & Co., Inc.",88.50,87.305000,1327500,None,None,None


Calculating averages for the change in price of the stock in 24h, 1 week and 1 month after the filing date.

In [18]:
def calculate_averages(df):
    # Convert percentage strings to numbers
    def convert_percentage(x):
        if pd.isna(x):
            return None
        return float(x.strip('%').replace('+', ''))
    
    # Convert percentages to numbers for all columns
    h24_changes = df['24h_Change_%'].apply(convert_percentage)
    w1_changes = df['1w_Change_%'].apply(convert_percentage)
    m1_changes = df['1m_Change_%'].apply(convert_percentage)
    
    # Calculate averages
    h24_avg = h24_changes.mean()
    w1_avg = w1_changes.mean()
    m1_avg = m1_changes.mean()
    
    # Calculate win rates (excluding None values)
    h24_winrate = (h24_changes[h24_changes.notna()] > 0).mean() * 100 if not h24_changes.empty else None
    w1_winrate = (w1_changes[w1_changes.notna()] > 0).mean() * 100 if not w1_changes.empty else None
    m1_winrate = (m1_changes[m1_changes.notna()] > 0).mean() * 100 if not m1_changes.empty else None
    
    print("Average Changes:")
    print(f"24 Hours: {'+' if h24_avg >= 0 else ''}{h24_avg:.2f}% (Win Rate: {h24_winrate:.1f}%)")
    if not pd.isna(w1_avg):
        print(f"1 Week:   {'+' if w1_avg >= 0 else ''}{w1_avg:.2f}% (Win Rate: {w1_winrate:.1f}%)")
    if not pd.isna(m1_avg):
        print(f"1 Month:  {'+' if m1_avg >= 0 else ''}{m1_avg:.2f}% (Win Rate: {m1_winrate:.1f}%)")

# Execute the analysis
analysis_df = calculate_price_changes()
calculate_averages(analysis_df)

Average Changes:
24 Hours: +1.13% (Win Rate: 66.7%)
